# In this notebook, we're going to cover some of the most fundamental concepts of tensors using TensorFlow

More specifically, we're going to cover:
* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & NumPy
* Using @tf.function (a way to speed up your regular Python functions)
* Using GPUs with Tensorflow (or TPUs)
* Exercises to try for yourself!


## Introduction to Tensors

In [5]:
# Import TensorFlow
import tensorflow as tf
print(tf.__version__)

2.14.0


In [6]:
tf.sysconfig.get_build_info()

OrderedDict([('cpu_compiler', '/usr/lib/llvm-16/bin/clang'),
             ('cuda_compute_capabilities',
              ['sm_35', 'sm_50', 'sm_60', 'sm_70', 'sm_75', 'compute_80']),
             ('cuda_version', '11.8'),
             ('cudnn_version', '8'),
             ('is_cuda_build', True),
             ('is_rocm_build', False),
             ('is_tensorrt_build', True)])

In [7]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [8]:
# Check the number of dimensions of a tensor (ndim stands for number of dimensions)
scalar.ndim

0

In [9]:
# Create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [10]:
# Check the dimension of our vector
vector.ndim

1

In [11]:
# Create a matrix (has mor than 1 dimension)
matrix = tf.constant([[10, 7],
                     [7, 10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [12]:
matrix.ndim

2

In [13]:
# Create another matrix
another_matrix = tf.constant([[10., 7.,],
                              [3., 2.],
                              [8., 9.]], dtype=tf.float16) # specify the data type with dtype parameter
list(another_matrix)

[<tf.Tensor: shape=(2,), dtype=float16, numpy=array([10.,  7.], dtype=float16)>,
 <tf.Tensor: shape=(2,), dtype=float16, numpy=array([3., 2.], dtype=float16)>,
 <tf.Tensor: shape=(2,), dtype=float16, numpy=array([8., 9.], dtype=float16)>]

In [14]:
# Check the dimension of another vector
another_matrix.ndim

2

In [15]:
# Create tensor
tensor = tf.constant([[[1, 2, 3], [4, 5, 6]],
                      [[7, 8, 9], [10, 11, 12]],
                      [[13, 14, 15], [16, 17, 17]]
                      ])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 17]]], dtype=int32)>

In [16]:
tensor.ndim

3

What we've created so far:
* Scalar: a single number
* Vector: a number with direction (e.g. wind speed and direction)
* Matrix: a 2-dimensional array of numbers
* Tensor: an n-dimensional array of numbers (when n can be any number, a 0-dimensional tensor is a scalar, a 1-dimensional tensor is a vector)

### Creating tensors with tf.Variable

In [17]:
# Create the same tensor with tf.Variable as above
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [18]:
# Let's try change one of the elements in our tensor
# changeable_tensor[0] = 7

In [19]:
# How about we try .assign()
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [20]:
# Let's try change our unchangable tensor
# unchangeable_tensor[0].assign(7)
unchangeable_tensor

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>

### Creating random tensors

Random tensors are tensors of some abitraty size which contain random numbers

In [21]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(7) # set seed for reproducibility
random_1 = random_1.normal(shape=(3, 2))
random_2 = tf.random.Generator.from_seed(7) # set seed for reproducibility
random_2 = random_2.normal(shape=(3, 2))

# Are they equal?
random_1, random_1, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.2878567 ],
        [-0.87579006, -0.08857017],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.2878567 ],
        [-0.87579006, -0.08857017],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

### Shuffle the order of elements in a tensorm

In [22]:
# Shuffle a tensor (valuable for when you want to shuffle your data so the inherent order doesn't effect learning)
not_shuffled = tf.constant([[10, 7],
                            [3, 4],
                            [2, 5]])
# Shuffle it
# tf.random.set_seed(123)
shuffled = tf.random.shuffle(not_shuffled, seed=1212)

In [23]:
shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 2,  5],
       [ 3,  4]], dtype=int32)>

Its interactions with operation-level seeds is as follows:

* If neither the global seed nor the operation seed is set: A randomly picked seed is used for this op.
* If the global seed is set, but the operation seed is not: The system deterministically picks an operation seed in conjunction with the global seed so that it gets a unique random sequence. Within the same version of tensorflow and user code, this sequence is deterministic. However across different versions, this sequence might change. If the code depends on particular seeds to work, specify both global and operation-level seeds explicitly.
* If the operation seed is set, but the global seed is not set: A default global seed and the specified operation seed are used to determine the random sequence.
* If both the global and the operation seed are set: Both seeds are used in conjunction to determine the random sequence.


In [24]:
tf.random.set_seed(1234)

@tf.function
def f():
    a = tf.random.uniform([1])
    b = tf.random.uniform([1])
    return a, b

@tf.function
def g():
    a = tf.random.uniform([1])
    b = tf.random.uniform([1])
    return a, b

print(f())  # prints '(A1, A2)'
print(g())  # prints '(A1, A2)'

(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.13047123], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1689806], dtype=float32)>)
(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.13047123], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1689806], dtype=float32)>)


In [25]:
@tf.function
def foo():
    a = tf.random.uniform([1], seed=1)
    b = tf.random.uniform([1], seed=1)
    return a, b
print(foo())  # prints '(A1, A1)'
print(foo())  # prints '(A2, A2)'

@tf.function
def bar():
    a = tf.random.uniform([1])
    b = tf.random.uniform([1])
    return a, b
print(bar())  # prints '(A1, A2)'
print(bar())  # prints '(A3, A4)'

(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1689806], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1689806], dtype=float32)>)
(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.7539084], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.7539084], dtype=float32)>)
(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.13047123], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1689806], dtype=float32)>)
(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.6087816], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.7539084], dtype=float32)>)


### Other ways to make tensors

In [26]:
# Create a tensor of all ones
tf.ones([10, 7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [27]:
# Create a tensor of all zeroes
tf.zeros(shape=[2, 3, 3])

<tf.Tensor: shape=(2, 3, 3), dtype=float32, numpy=
array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]], dtype=float32)>

### Turn NumPy arrays into tensors

The main difference between NumPy arrays and TensorFlow tensors is that tensors can be run on a GPU

In [28]:
# Numpy arrays -> tensors
import numpy as np
numpy_A = np.arange(1, 25, dtype=np.int32)

# x = tf.constant(some_metrix) # capital for matrix or tensor
# y = tf.constant(vector) # non-capital for vector

In [29]:
A = tf.constant(numpy_A, shape=(2, 3, 4))
B = tf.constant(numpy_A)
A, B

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>)

In [30]:
2 * 3 * 4

24

In [31]:
3 * 8

24

### Getting info from tensors
When dealing with tensors you probably want to be aware of the following attributes:
* Shape
* Rank
* Axis or dimension
* Size

In [32]:
# Create a rank 4 tensor (4 dimensions)
rank_4_tensor = tf.zeros(shape=(2, 3, 4, 5))
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [33]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [34]:
# Get various attributes of our tensor
print("Datatype of every element:", rank_4_tensor.dtype)
print("Number of dimensions (rank):", rank_4_tensor.ndim)
print("Shape of tensor:", rank_4_tensor.shape)
print("Number along the 0 axis:", rank_4_tensor.shape[0])
print("Elements along the last axis", rank_4_tensor.shape[-1])
print("Total number of elements:", tf.size(rank_4_tensor).numpy())

Datatype of every element: <dtype: 'float32'>
Number of dimensions (rank): 4
Shape of tensor: (2, 3, 4, 5)
Number along the 0 axis: 2
Elements along the last axis 5
Total number of elements: 120


### Indexing tensors

Tensors can be indexed like Python list

In [35]:
some_list = [1, 2, 3, 4]
some_list[:2]

[1, 2]

In [36]:
# Get the first 2 elements of each dimension
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [37]:
# Get the first element from each dimension from each index except for the final one
rank_4_tensor[:1, :1, :1, :]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [38]:
# Create a rank 2 tensor (2 dimensions)
rank_2_tensor = tf.constant([
    [1,2],
    [3, 4]
])
rank_2_tensor, rank_2_tensor.shape, rank_2_tensor.ndim

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4]], dtype=int32)>,
 TensorShape([2, 2]),
 2)

In [39]:
# Get the last item of each of our rank 2 tensor
rank_2_tensor[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([2, 4], dtype=int32)>

In [40]:
# Adding extra dimension for our rank 2 tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis] # ... = :, :, (means everything before)
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[1],
        [2]],

       [[3],
        [4]]], dtype=int32)>

In [41]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=-1) # "-1" means expand the final axis

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[1],
        [2]],

       [[3],
        [4]]], dtype=int32)>

In [42]:
# Expand the 0-axis
tf.expand_dims(rank_2_tensor, axis=0)

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[1, 2],
        [3, 4]]], dtype=int32)>

### Manipulating tensors (tensor operations)

**Basic operations**

`+`, `-`, `*`, `/`

In [43]:
# You can add values to a tensor using the addiction operator
tensor = tf.constant([[10, 7],
                      [3, 4]])
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [44]:
# Original tensor is unchanged
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [45]:
# Multiplication also works
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [46]:
# Subtraction if you want
tensor - 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]], dtype=int32)>

In [47]:
# We can use the tensorflow built-in functions too
# !!! tf.multiply, etc is better than * 10!!!
print("Tensor + C:", tf.add(tensor, 10).numpy())
print("Tensor - C:", tf.subtract(tensor, 10).numpy())
print("Tensor * C:", tf.multiply(tensor, 10).numpy())
print("Tensor / C:", tf.divide(tensor, 10).numpy())

Tensor + C: [[20 17]
 [13 14]]
Tensor - C: [[ 0 -3]
 [-7 -6]]
Tensor * C: [[100  70]
 [ 30  40]]
Tensor / C: [[1.  0.7]
 [0.3 0.4]]


**Matrix multiplication**

In machine learning, matrix is one of the most tensor operation

There are two rules our tensors(or matrices) need to fulfill if we're going to matrix multiply them:
1. The width of X tensor must equal the length of Y tensor
2. The resulting matrix has the shape of the inner dimensions

In [48]:
# Matrix multiplication
tf.matmul(tensor, tensor)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [49]:
# mini homework

In [50]:
a_tensor = tf.constant([
    [1, 2, 5],
    [7, 2, 1],
    [3, 3, 3]
])
b_tensor = tf.constant([
    [3, 5],
    [6, 7],
    [1, 8]
])

print(tf.matmul(a_tensor, b_tensor).numpy())
print()
print((a_tensor @ b_tensor).numpy())

[[20 59]
 [34 57]
 [30 60]]

[[20 59]
 [34 57]
 [30 60]]


In [51]:
# Create 2 shape=(3, 2) tensors and waiting for error ;)
X = tf.constant([[1, 2],
                 [3, 4],
                 [5, 6]])
Y = tf.constant([[7, 8],
                 [9, 10],
                 [11, 12]])
# X @ Y


In [52]:
# Reshape Y
tf.reshape(Y, shape=(2, 3))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 7,  8,  9],
       [10, 11, 12]], dtype=int32)>

In [53]:
# Try to matrix multiply X by reshaped Y
X @ tf.reshape(Y, shape=(2, 3))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [54]:
# Now reshape X for multiplying with Y
tf.matmul(tf.reshape(X, shape=(2, 3)), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 58,  64],
       [139, 154]], dtype=int32)>

In [55]:
tf.reshape(X, shape=(2, 3)).shape, Y.shape

(TensorShape([2, 3]), TensorShape([3, 2]))

In [56]:
# Can do the same with transpose
X, tf.transpose(X), tf.reshape(X, shape=(2, 3))

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 3, 5],
        [2, 4, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]], dtype=int32)>)

In [57]:
# Try matrix multiplication with transpose rather than reshape
tf.matmul(tf.transpose(X), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

**The dot product**

Matrix multiplication is also referred to as the dot product.

You can perform matrix multiplication using:
* `tf.matmul()`
* `tf.tensordot()`
* `@`

In [58]:
# Perform the dot product on X and Y (requires X or Y to be transposed)
tf.tensordot(tf.transpose(X), Y, axes=1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

In [59]:
# Perform matrix multiplication between X and Y (transposed)
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

In [60]:
# Perform matrix multiplication between X and Y (reshaped)
tf.matmul(X, tf.reshape(Y, shape=(2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [61]:
# Check the values of Y, reshaped Y and transposed Y
print("Normal YL:")
print(Y.numpy(), '\n')

print("Y reshaped to (2, 3):")
print(tf.reshape(Y, (2, 3)).numpy(), "\n")

print("Y transposed to (2, 3):")
print(tf.transpose(Y).numpy())

Normal YL:
[[ 7  8]
 [ 9 10]
 [11 12]] 

Y reshaped to (2, 3):
[[ 7  8  9]
 [10 11 12]] 

Y transposed to (2, 3):
[[ 7  9 11]
 [ 8 10 12]]


In [62]:
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

Generally, when perfoming matrix multiplication on two tensors and one of the axes doesn't line up, you will tranpose (rather than reshape) one of the tensors to get satisfy the matrix multiplication rules.

### Changing the datatype of a tensor

In [63]:
tf.__version__

'2.14.0'

In [64]:
# Create a new tensor with default datatype (float32)
B = tf.constant([1.7, 7.4])
B, B.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.7, 7.4], dtype=float32)>,
 tf.float32)

In [65]:
C = tf.constant([7, 10])
C, C.dtype

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10], dtype=int32)>,
 tf.int32)

In [66]:
# Change from float32 to float16 (reduce precision)
D = tf.cast(B, dtype=tf.float16)
D, D.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 7.4], dtype=float16)>,
 tf.float16)

In [67]:
# Change from int32 to int16
E = tf.cast(C, dtype=tf.int16)
E, E.dtype

(<tf.Tensor: shape=(2,), dtype=int16, numpy=array([ 7, 10], dtype=int16)>,
 tf.int16)

In [68]:
E_float16 = tf.cast(E, dtype=tf.float16)
E, E.dtype

(<tf.Tensor: shape=(2,), dtype=int16, numpy=array([ 7, 10], dtype=int16)>,
 tf.int16)

### Aggregating tensors

Aggregating tensors = condensing them from mupltiple values down to a smaller amount of values

In [69]:
# Get the absolute values
D = tf.constant([-6, 10])
E = tf.abs(D)
E

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 6, 10], dtype=int32)>

Let's go through the following forms of aggregation:
* Get the minimum
* Get the maximum
* Get the mean
* Get the sum
* Get the prod

In [70]:
# Create a random tensor with values between 0 and 100 of size 50
E = tf.constant(np.random.randint(0, 100, size=50))
E

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([95, 58, 98, 48, 96, 91, 54, 60, 73, 21,  6, 41, 48, 25, 67,  9, 42,
       94, 61, 85,  4, 94, 23,  9, 11, 24, 27, 79, 58,  7, 86, 21, 74, 26,
        1, 91, 96, 23, 51, 10, 98, 20, 37, 83, 81, 57, 14, 87, 28,  1])>

In [71]:
# Get the minimum of a tensor
tf.math.reduce_min(E)

<tf.Tensor: shape=(), dtype=int64, numpy=1>

In [72]:
# Get the maximum of a tensor
tf.math.reduce_max(E)

<tf.Tensor: shape=(), dtype=int64, numpy=98>

In [73]:
# Get the mean of a tensor
tf.math.reduce_mean(E)

<tf.Tensor: shape=(), dtype=int64, numpy=49>

In [74]:
# Get the sum of a tensor
tf.math.reduce_sum(E)

<tf.Tensor: shape=(), dtype=int64, numpy=2493>

In [75]:
# Change tensor to a real
real_E = tf.cast(E, dtype=tf.float32)

In [76]:
# Get the STD (Standart Deviation)
tf.math.reduce_std(real_E)

<tf.Tensor: shape=(), dtype=float32, numpy=32.37963>

In [77]:
# Get the Variance
tf.math.reduce_variance(real_E)

<tf.Tensor: shape=(), dtype=float32, numpy=1048.4404>

In [78]:
# Get the prod
tf.math.reduce_prod(E)

<tf.Tensor: shape=(), dtype=int64, numpy=-3982263990037250048>

**Find the positional maximum and minimum**

In [79]:
# Create a new tensor for finding the positional maximum and minimum
tf.random.set_seed(42)
F = tf.random.uniform(shape=[50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [80]:
# Find the positional minimum
min_index = tf.math.argmin(F).numpy()
min_index, F[min_index].numpy()

(16, 0.009463668)

In [81]:
# Find the positional maximum
max_index = tf.math.argmax(F).numpy()
max_index, F[max_index].numpy()

(42, 0.9671384)

In [82]:
# Check the equality of tf.argmax and tf.reduce_max
(F[tf.math.argmax(F)] == tf.math.reduce_max(F)).numpy(), tf.math.reduce_max(F)

(True, <tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>)

In [83]:
# Check the equality of tf.argmin and tf.reduce_min
(F[tf.math.argmin(F)] == tf.math.reduce_min(F)).numpy(), tf.math.reduce_min(F)

(True, <tf.Tensor: shape=(), dtype=float32, numpy=0.009463668>)

### Squeezing a tensor (removing all single dimensions)

In [84]:
tf.random.set_seed(12)
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1, 1, 1, 1, 50))
G

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.47720265, 0.33357763, 0.4960823 , 0.48351312, 0.2963959 ,
           0.69444907, 0.68488   , 0.6662284 , 0.07551682, 0.06608915,
           0.91428673, 0.21982336, 0.066468  , 0.37987065, 0.88209164,
           0.73124087, 0.59181285, 0.9785874 , 0.0937705 , 0.37701082,
           0.2055552 , 0.2800591 , 0.44172525, 0.9199766 , 0.49873018,
           0.10266078, 0.12177575, 0.10520744, 0.22547913, 0.39156199,
           0.5587522 , 0.7095549 , 0.04214132, 0.03028095, 0.14591146,
           0.7897457 , 0.52633774, 0.8487345 , 0.08758485, 0.5727763 ,
           0.74252594, 0.63061965, 0.03749514, 0.0432719 , 0.13088238,
           0.6540686 , 0.9438777 , 0.65461504, 0.10520577, 0.8353534 ]]]]],
      dtype=float32)>

In [85]:
G.shape

TensorShape([1, 1, 1, 1, 50])

In [86]:
G_squeezed = tf.squeeze(G)
G_squeezed, G_squeezed.shape

(<tf.Tensor: shape=(50,), dtype=float32, numpy=
 array([0.47720265, 0.33357763, 0.4960823 , 0.48351312, 0.2963959 ,
        0.69444907, 0.68488   , 0.6662284 , 0.07551682, 0.06608915,
        0.91428673, 0.21982336, 0.066468  , 0.37987065, 0.88209164,
        0.73124087, 0.59181285, 0.9785874 , 0.0937705 , 0.37701082,
        0.2055552 , 0.2800591 , 0.44172525, 0.9199766 , 0.49873018,
        0.10266078, 0.12177575, 0.10520744, 0.22547913, 0.39156199,
        0.5587522 , 0.7095549 , 0.04214132, 0.03028095, 0.14591146,
        0.7897457 , 0.52633774, 0.8487345 , 0.08758485, 0.5727763 ,
        0.74252594, 0.63061965, 0.03749514, 0.0432719 , 0.13088238,
        0.6540686 , 0.9438777 , 0.65461504, 0.10520577, 0.8353534 ],
       dtype=float32)>,
 TensorShape([50]))

### One-hot encoding tensors

In [87]:
# Create a list of indices
some_list = [0, 1, 2, 3] # could be red, green, blue and purple

# One-hot encode our list of indices
tf.one_hot(some_list, depth=4)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [88]:
# Specify custom values for one hot encoding
tf.one_hot(some_list, depth=4, on_value="Samsung", off_value=f"Poco")

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'Samsung', b'Poco', b'Poco', b'Poco'],
       [b'Poco', b'Samsung', b'Poco', b'Poco'],
       [b'Poco', b'Poco', b'Samsung', b'Poco'],
       [b'Poco', b'Poco', b'Poco', b'Samsung']], dtype=object)>

#### Practice

In [89]:
n = 10
sequence = tf.range(n)
sequence

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [90]:
one_hot_tensor = tf.one_hot(sequence, depth=n)

tf.cast(one_hot_tensor, dtype=tf.int8)

<tf.Tensor: shape=(10, 10), dtype=int8, numpy=
array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int8)>

### Math functions:
* Squaring
* Natural logarithm
* Square root
* Trigonometric functions
* Round
* Reduce_any
* Reduce all

In [91]:
# Create a new tensor
H = tf.range(1, 10)
H_float16 = tf.cast(H, dtype=tf.float16)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [92]:
# Square it
tf.math.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>

In [93]:
# Find the square root (!!! method requires no-int dtype)
tf.math.sqrt(H_float16)

<tf.Tensor: shape=(9,), dtype=float16, numpy=
array([1.   , 1.414, 1.732, 2.   , 2.236, 2.45 , 2.646, 2.828, 3.   ],
      dtype=float16)>

In [94]:
# Find a natural logarithm
tf.math.log(H_float16)

<tf.Tensor: shape=(9,), dtype=float16, numpy=
array([0.    , 0.6934, 1.099 , 1.387 , 1.609 , 1.792 , 1.946 , 2.08  ,
       2.197 ], dtype=float16)>

In [95]:
# sin, cos, tan tensor
tf.math.sin(H_float16), tf.math.cos(H_float16), tf.math.tan(H_float16)

(<tf.Tensor: shape=(9,), dtype=float16, numpy=
 array([ 0.8413,  0.909 ,  0.1411, -0.757 , -0.959 , -0.2793,  0.657 ,
         0.9893,  0.412 ], dtype=float16)>,
 <tf.Tensor: shape=(9,), dtype=float16, numpy=
 array([ 0.5405, -0.4163, -0.99  , -0.654 ,  0.2837,  0.96  ,  0.754 ,
        -0.1455, -0.911 ], dtype=float16)>,
 <tf.Tensor: shape=(9,), dtype=float16, numpy=
 array([ 1.558 , -2.186 , -0.1426,  1.158 , -3.38  , -0.291 ,  0.8716,
        -6.8   , -0.4524], dtype=float16)>)

In [96]:
# Round tensor
unrounded_tensor = tf.constant([0.4, 0.7, 4.3, 6.6, float("inf")])
tf.round(unrounded_tensor)

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([ 0.,  1.,  4.,  7., inf], dtype=float32)>

In [97]:
# reduce_any, reduce_all
# (reduce_any = x or y, reduce all = x and y)
N = tf.constant([[True, True],
                 [False, False]])
tf.math.reduce_any(N).numpy(), tf.math.reduce_all(N).numpy()

(True, False)

In [98]:
# pow
x = tf.constant([[2, 2], [3, 3]])
y = tf.constant([[8, 16], [2, 3]])
tf.pow(x, y)


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[  256, 65536],
       [    9,    27]], dtype=int32)>

### Tensors and NumPy

TensorFlow interacts beatufilly with NumPy arrays.

**One of the main differences between a TensorFlow tensor and NumPy tensor is that a TensorFlow tensor can be run on a GPU or on a TPU**

In [99]:
# Create a tensor directly from a NumPy array
J = tf.constant(np.array([3., 7., 10.]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [100]:
# Convert our tensor back to a NumPy array
np.array(J), type(np.array(J)), J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray, array([ 3.,  7., 10.]), numpy.ndarray)

In [101]:
# The default types of each are slightly different
numpy_J = tf.constant(np.array([3., 7., 10.]))
tensor_J = tf.constant([3., 7., 10.])
# Check the datatypes of each
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

### Finding accesss to GPU's

In [102]:
tf.config.list_physical_devices() # 'GPU'

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [103]:
!nvidia-smi

Sun Dec 10 17:54:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    29W /  70W |    521MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

> **Note:** if you have access to a CUDA-enabled GPU, TensorFlow will automatically use it whenever possible.

## 00. TensorFlow Fundamentals Exercises

In [104]:
scalar = tf.constant(5)
vector = tf.constant([1, 2])
vector = tf.constant([[1, 2], [3, 4]])
vector = tf.constant([[1, 2, 3], [4, 5, 6]])

In [105]:
vector.shape, vector.ndim, tf.size(vector)

(TensorShape([2, 3]), 2, <tf.Tensor: shape=(), dtype=int32, numpy=6>)

In [106]:
K = tf.random.uniform(shape=[5, 300])
P = tf.random.uniform(shape=[5, 300])

In [107]:
tf.matmul(K, tf.reshape(P, shape=(300, 5)))

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[81.2483  , 77.67445 , 77.35405 , 78.526505, 79.57772 ],
       [78.166985, 73.08165 , 75.13778 , 75.07493 , 69.7845  ],
       [73.71886 , 70.23656 , 69.990814, 70.96835 , 70.85155 ],
       [72.870674, 71.42348 , 75.63077 , 75.57718 , 72.66597 ],
       [79.546394, 75.29173 , 78.197235, 80.91321 , 80.82392 ]],
      dtype=float32)>

In [108]:
X = tf.tensordot(K, P, axes=0)

In [109]:
L = tf.random.uniform(shape=[224, 224, 3])

In [110]:
# min and max
tf.math.reduce_min(L), tf.math.reduce_max(L)

(<tf.Tensor: shape=(), dtype=float32, numpy=3.5762787e-07>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.99999905>)

In [111]:
# Squeeze
H = tf.random.uniform(shape=[1, 224, 224, 3])
H = tf.squeeze(H)

In [112]:
# Index of the max
F = tf.range(1, 11)
tf.math.argmax(F)

<tf.Tensor: shape=(), dtype=int64, numpy=9>

In [113]:
# One-hot tensor
tf.one_hot(F, depth=10)

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>